# Split-Apply-Combine operations with `.groupby()`

Our data often consists of individual observations or events. To make sense of the patterns in the data it can be helpful to aggregate the data within categorical groups. The general method is described as the Split-Apply-Combine strategy for data analysis, as described in [this classic paper by Hadley Wickham](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf).

- In SQL this is done with `GROUPBY`
- In Tableau this is done with all visualizations + Level of Detail (LOD) calculations
- In R this is done with the `dplyr` package
- **In Python with Pandas this is done with `.groupby()`**

## Groupby

From [the Pandas `.groupby()` documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html):

**By “group by” we are referring to a process involving one or more of the following steps:**

- **Splitting** the data into groups based on some criteria
- **Applying** a function to each group independently
- **Combining** the results into a data structure

After splitting, in **the apply step**, we do something to the groups, like:

- **Aggregation**: compute a summary statistic (or statistics) for each group, like group sums or means, or group sizes / counts. ***Reduces the number of rows to one per group***
- **Transformation**: perform some group-specific computations and return a like-indexed object, such as a standardize data (zscore) within a group, or filling NAs within groups with a value derived from each group. ***Keeps the number of rows the same (e.g. a new column with the group average in each row for easy comparisons to the individuals)***
- **Filtration**: discard some groups, according to a group-wise computation that evaluates True or False, such as discarding data that belongs to groups with only a few members, or filtering out data based on the group sum or mean


### Graphical Example 

As Jake VanderPlas shows in the 
[Aggregation and Grouping](http://nbviewer.jupyter.org/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.08-Aggregation-and-Grouping.ipynb) 
section of his excellent 
[Python Data Science Handbook](https://github.com/jakevdp/PythonDataScienceHandbook)
, an archtypical example of a `groupby()` operation with a sum aggregation is:

<img src='images/split-apply-combine.svg' width=600>

---

*To preserve the mystery, select from the notebook menus*

`Edit -> Clear All Outputs`

---

## Pandas groupby syntax

Let's try just a slightly more complex example to familiarize ourselves with the Pandas syntax for the aggregation and transformation operations. **To see some very common behaviors, we need multiple numerical columns.** *(We'll save multiple categorical columns for later...)*

In [1]:
import pandas as pd

### Create the example DataFrame

In [2]:
key_list = ['A','B','C','A','B','C']
data1_list = [2, 4, 6, 8, 10, 12]
data2_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

df = pd.DataFrame({'key':key_list, 'data1':data1_list, 'data2':data2_list})
df

,key,data1,data2
0,A,2,0.1
1,B,4,0.2
2,C,6,0.3
3,A,8,0.4
4,B,10,0.5
5,C,12,0.6


### The operation creates a GroupBy object

In [3]:
group = df.groupby('key')
group

### The individual rows in each group are still separate!

In [4]:
group.groups

{'A': Int64Index([0, 3], dtype='int64'),
 'B': Int64Index([1, 4], dtype='int64'),
 'C': Int64Index([2, 5], dtype='int64')}

### We can access the groups if we need to

In [5]:
group.get_group('A')

,key,data1,data2
0,A,2,0.1
3,A,8,0.4


---

## Aggregate


There are a few different variations of the aggregate syntax. The excellent blog post 
[Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428) 
by Ted Petrou talks about the trade-offs. I will first show you a shortcut method, but in general I try to follow his suggestions for preferred syntax.

- `.agg()` is just short for `.aggregate()` and is fine to use
- **Aggregate results in a single combined row for each group**
- **You can specify multiple statistical functions at once with a list**

Some [descriptive statistics are built into Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#descriptive-statistics) and it's easy to import Numpy and use functions from there, too.

### This "function name only" shortcut syntax is not recommended

But it does work fine for a quick result

In [6]:
df.groupby('key').agg('sum')

,data1,data2
key,,
A,10,0.5
B,14,0.7
C,18,0.9


### Multiple aggregation functions list results in multi-index columns

- You can supply a list of functions
- The multi-index combines the original column name with the aggregation function that was applied

In [7]:
df.groupby('key').agg(['sum','mean'])

data1      data2      
      sum mean   sum  mean
key                       
A      10    5   0.5  0.25
B      14    7   0.7  0.35
C      18    9   0.9  0.45

## Preferred aggregate syntax

Because in many cases we have multiple data columns, **the preferred aggregation syntax specifies both the name of the data column and the function(s) to be applied in a dictionary format**

`df.groupby('grouping column').agg({'aggregating column':'aggregating function'})`

In [8]:
df.groupby('key').agg({'data1':'sum'})

,data1
key,
A,10
B,14
C,18


### Easy to apply different functions to different columns

Notice we don't retain the name of the applied function with only one function per column...

In [9]:
df.groupby('key').agg({'data1':'sum', 'data2':'mean'})

,data1,data2
key,,
A,10,0.25
B,14,0.35
C,18,0.45


---

**To try the exercise below, select this cell and from the Jupyter menus choose**

`Run -> Run All Above Selected Cell`

## EXERCISE

**Use the preferred syntax to find the maximum and minimum simultaneously on only the data2 column**

*Note: Type instead of using copy/paste for better retention*

*Expected output*

<img src='images/data2_max_min.png'>

---

## Transform

**You'll probably use aggregation more often, but it's important to know how to transform.** 

- Applies a function to each of the groups
- Only a single function is allowed – it will be applied to all the columns
- **Doesn't change the number of rows from the original!**
- *Notice that we lost the 'key' column, but we retain the Index!*

In [10]:
df.groupby('key').transform('mean')

,data1,data2
0,5,0.25
1,7,0.35
2,9,0.45
3,5,0.25
4,7,0.35
5,9,0.45


### Transform into a new column

More commonly you'll create a new column out of the transformation results so you can compare individuals to the group result.

**If you want a single column (Series) out, you have to either**

- select a single column from the groupby object to pass to the transform function
- select a single column from the transform output

**using the standard "name of column in square brackets" notation**

In [11]:
df.groupby('key')['data1'].transform('mean')

0    5
1    7
2    9
3    5
4    7
5    9
Name: data1, dtype: int64

In [12]:
df.groupby('key').transform('mean')['data1']

0    5
1    7
2    9
3    5
4    7
5    9
Name: data1, dtype: int64

### Create that new column

In [13]:
df['data1_mean'] = df.groupby('key')['data1'].transform('mean')
df

,key,data1,data2,data1_mean
0,A,2,0.1,5
1,B,4,0.2,7
2,C,6,0.3,9
3,A,8,0.4,5
4,B,10,0.5,7
5,C,12,0.6,9


### Create boolean column for above group mean

In [14]:
df['d1_above_mean'] = df['data1'] > df['data1_mean']
df

,key,data1,data2,data1_mean,d1_above_mean
0,A,2,0.1,5,False
1,B,4,0.2,7,False
2,C,6,0.3,9,False
3,A,8,0.4,5,True
4,B,10,0.5,7,True
5,C,12,0.6,9,True


---

**To try the exercise below, select this cell and from the Jupyter menus choose**

`Run -> Run All Above Selected Cell`

### EXERCISE: Transform group data2 minimum into a new `data2_min` column

*Note: Type instead of using copy/paste for better retention*

*Expected output*

<img src='images/data2_min_col.png'>

### EXERCISE: Create a new `d2_equals_min` column showing data2 value == data2 minimum

*Expected output*

<img src='images/d2_equals_min_col.png'>

---